In [48]:
import pandas as pd
import plotly.express as px
import random
mpt = 'pk.eyJ1IjoiamFjb3BvZ3VhbmV0dGkiLCJhIjoiY2syemcwYXc2MGVkMTNjcjNzN2diYTZ1byJ9.nwzs14NIRaUTmvaj8yeLSA'
px.set_mapbox_access_token(mpt)
#df = px.data.carshare()
# df = pd.DataFrame()
lat=[]
lon=[]

# speed should be km/kwh
# in Kona 415/64=6.48 km/kWh when on a segment duration : distance in m/speed
#first node id:
#lat="39.0971725" lon="-123.2097626
#second node id:
# lat="39.0945650" lon="-123.2077710"
with open("../worldcities.csv","r") as file:
    data = file.readlines()
    for line in data:
        values = line.split(",")
        #if values[4]=='India':
        if (values[4] == 'United States') and (values[7] == 'California'):
            lat.append(float(values[2]))
            lon.append(float(values[3]))
df = pd.DataFrame()
df['lat']=lat
df['lon']=lon
# print(df)

from utils import india_route
start=random.randrange(len(df['lat']))
#start=100
slat = df['lat'][start]
slon = df['lon'][start]
# first node id
#slat = 39.0945650
#slon = -123.2077710
#slat = 39.4499503
#slon = -123.8045130
# second node id
df['lat'][0]= 39.0971725
df['lon'][0]=-123.2097626
#df['lat'][0]= 39.4501070
#df['lon'][0]=-123.8045149
#449.60

print("Source {},{}".format(slat,slon))
distance_json = india_route.get_table(slat, slon, df['lat'], df['lon'])    
m_df = pd.DataFrame()
destinations = distance_json['destinations']
m_df['distance'] = [distance/1000.0 for distance in distance_json['distances'][0]]
m_df['duration'] = [duration/3600.0 for duration in distance_json['durations'][0]]
m_df['speed'] = [(distance/1000.0)/(duration/3600.0+0.0001) for distance,duration in zip(distance_json['distances'][0],distance_json['durations'][0])]
m_df['lat'] = [dest['location'][1] for dest in destinations]
m_df['lon'] = [dest['location'][0] for dest in destinations]
polypoints=[]
kwh=20
for duration,dest in zip(distance_json['durations'][0],destinations):
    if duration < kwh*3600.0:
        polypoints.append([dest['location'][1],dest['location'][0]])

#m_df['color'] = ['< 300 km' if distance < 300.0 else 'far' for distance in m_df['distance']]
m_df['color'] = ['< '+str(kwh)+' kWh' if duration < kwh else 'far' for duration in m_df['duration']]
m_df['size'] = [1 for i in range(len(m_df['lat']))]
#m_df = m_df.append(pd.DataFrame([[0.0, slat, slon, 'source',6]],columns=['distance','lat','lon','color','size']),ignore_index=True)
#m_df = m_df.sort_values('distance')
m_df = m_df.append(pd.DataFrame([[0.0, slat, slon, 'source',6]],columns=['duration','lat','lon','color','size']),ignore_index=True)
m_df = m_df.sort_values('duration')
fig = px.scatter_mapbox(m_df, lat="lat", lon="lon", color="color",size='size', hover_name='speed',zoom=3)

##############################################
# Contouring Needs unigorm grid!
if 0:
    import conrec
    #make matrix for conrect function
    x=list(set(m_df['lat']))
    x.sort()
    y=list(set(m_df['lon']))
    y.sort(reverse=True)
    M=4*np.ones((len(x),len(y)))
    k=0
    for dest in destinations:
        i=np.where(np.asarray(x) == (dest['location'][1]))
        j=np.where(np.asarray(y) == (dest['location'][0]))
        M[i[0],j[0]]=m_df['duration'][k]
        k=k+1
    S=M.tolist()
    a=conrec.conrec(S, x, y, [2])
    print(a)
    latpoly2=[]
    lonpoly2=[]
    for p in a[0][1][0]:
        latpoly2.append(p[0])
        lonpoly2.append(p[1])
    points = a[0][1][0]
#24.8382706
#291.21
# Convex hull
import numpy as np
latpoly=[]
lonpoly=[]
from scipy.spatial import ConvexHull
#hull = ConvexHull(points)
hull = ConvexHull(np.asarray(polypoints))
for vrt in hull.vertices:
     latpoly.append(polypoints[vrt][0])
     lonpoly.append(polypoints[vrt][1])
latpoly.append(polypoints[hull.vertices[0]][0])
lonpoly.append(polypoints[hull.vertices[0]][1])
import plotly.graph_objects as go
# test if point s are all in there
#for k in range(len(lonpoly)-1):
#    fig.add_trace(go.Scattermapbox(
#        mode = "markers+lines",
#        #lon = [polypoints[k][1],polypoints[k+1][1]],
#        #lat = [polypoints[k][0],polypoints[k+1][0]],
#        lon = [lonpoly[k],lonpoly[k+1]],
#        lat = [latpoly[k],latpoly[k+1]],
#        marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = lonpoly,
    lat = latpoly,
    marker = {'size': 10}))
fig.show()


# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(m_df)

Source 37.9939,-122.3188


In [49]:
# Get some routes
from utils import india_route
for i in range(len(m_df.index)):
    if i < 4 and m_df['distance'][i] > 1.0:
        end=random.randrange(len(m_df['lat']))
        elat = m_df['lat'][i]
        elon = m_df['lon'][i]
        print("routing from {},{} to {},{} dist = {}".format(slat, slon, elat, elon, m_df['distance'][i]))
        route = india_route.get_route(slat,elat,slon,elon).json()
    #     print(route['routes'][0]['legs'])
        locations = []
        for leg in route['routes'][0]['legs']:
            for step in leg['steps']:
                for intersection in step['intersections']:
    #                 print(intersection['location'])
                    locations.append(intersection['location'])
        map_df = pd.DataFrame()
        map_df['lon'] = [location[0] for location in locations]
        map_df['lat'] = [location[1] for location in locations]
        print(map_df)
        fig = px.scatter_mapbox(map_df, lat="lat", lon="lon", zoom=4)
        fig.show()
    

routing from 37.9939,-122.3188 to 39.097173,-123.209763 dist = 182.467
-122.3188,37.9939
-123.209763,39.097173
[-122.318609, 37.994]
[-123.209763, 39.097173]
received
            lon        lat
0   -122.318609  37.994000
1   -122.319212  37.995790
2   -122.318328  37.995883
3   -122.318452  37.996637
4   -122.317589  37.996726
..          ...        ...
326 -123.195041  39.106102
327 -123.197698  39.105243
328 -123.200125  39.104342
329 -123.200508  39.104036
330 -123.209763  39.097173

[331 rows x 2 columns]
routing from 37.9939,-122.3188 to 33.579204,-117.728976 dist = 706.2791
-122.3188,37.9939
-117.728976,33.579204
[-122.318609, 37.994]
[-117.728976, 33.579204]
received
            lon        lat
0   -122.318609  37.994000
1   -122.318405  37.993455
2   -122.317234  37.993467
3   -122.316343  37.993467
4   -122.315397  37.993480
..          ...        ...
694 -117.726457  33.580692
695 -117.726667  33.580294
696 -117.729095  33.580211
697 -117.728732  33.580074
698 -117.728976  33.

In [50]:
# create a grid and use it for trip.
latlist = []
lonlist = []
for i in range(-7, 7):
    for j in range(-7, 7):
        latlist.append(slat + (.15 * i))
        lonlist.append(slon + (.15 * j))
        
grid_df = pd.DataFrame()
grid_df['lat'] = latlist
grid_df['lon'] = lonlist
distance_json = india_route.get_table(slat, slon, grid_df['lat'], grid_df['lon'])    
m_df = pd.DataFrame()
destinations = distance_json['destinations']
m_df['distance'] = [distance/1000.0 for distance in distance_json['distances'][0]]
m_df['duration'] = [duration/3600.0 for duration in distance_json['durations'][0]]
m_df['lat'] = [dest['location'][1] for dest in destinations]
m_df['lon'] = [dest['location'][0] for dest in destinations]
def get_color(distance):
    if distance < 100:
        return '< 100 km'
    elif distance < 150:
        return '< 150 km'
    else:
        return 'far'
m_df['color'] = [get_color(distance) for distance in m_df['distance']]
m_df['size'] = [.1 for i in range(len(m_df['lat']))]
m_df = m_df.append(pd.DataFrame([[0.0, slat, slon, 'source',2]],columns=['distance','lat','lon','color','size']),ignore_index=True)
m_df = m_df.sort_values('distance')
fig = px.scatter_mapbox(m_df, lat="lat", lon="lon", color="color", size="size", zoom=5)
fig.show()
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(m_df)


        

    